# Example for usage ARc Policy via a Strands Agent

Install requirements

In [1]:
%pip install -r requirements.txt --no-cache-dir --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.7/993.7 kB 221.7 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 372.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 438.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 466.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 120.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 537.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 418.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 533.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 685.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 415.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 155.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [2]:
# %pip install -r requirements.txt --upgrade

In [3]:
# If needed, this will allow for the restart of the notebook

# from IPython.display import display_html
# def restart_kernel():
#     display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)
# restart_kernel()


In [4]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [5]:
!pip freeze | grep boto
!pip freeze | grep agentcore

boto3==1.42.42
botocore==1.42.42
opentelemetry-instrumentation-boto==0.60b1
bedrock-agentcore==1.2.1
bedrock-agentcore-starter-toolkit==0.2.10


In [15]:
# Import libraries
import os
import json
import requests
import boto3
import time
from boto3.session import Session
from strands.tools import tool

# Get boto session
boto_session = Session()

## Setup for the agent

### Create Code for the Agent
Create agents folder if it's not created.

In [16]:
![ ! -d "agents" ] && mkdir agents

In [17]:
%%writefile agents/claimvalidation-agent.py
import os
import logging
import asyncio
from mcp import stdio_client, StdioServerParameters
from strands import Agent
from strands.models import BedrockModel
from strands.tools import tool
from strands.multiagent.a2a import A2AServer
from strands.tools.mcp import MCPClient
import argparse
from fastapi import FastAPI
import uvicorn
from strands.hooks import HookProvider, HookRegistry, MessageAddedEvent, BeforeModelCallEvent, BeforeToolCallEvent
from pydantic import BaseModel
from botocore.config import Config as BotocoreConfig
from strands.telemetry import StrandsTelemetry
from findings_utils import extract_reasoning_findings
from strands_tools import retrieve

# Standard library imports
import json
from datetime import datetime

# AWS SDK
import boto3

from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException



# Configure the root strands logger
# logging.getLogger("strands").setLevel(logging.DEBUG)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# Add a handler to see the logs
# logging.basicConfig(
#     format="%(levelname)s | %(name)s | %(message)s", 
#     handlers=[logging.StreamHandler()]
# )

# Setup tracing - commented out for now as this adds a lot of trace output that really isn't interesting
StrandsTelemetry().setup_console_exporter()

# NOTE: To send the OTEL data to an ADOT collector, additional exporter needs to be used




# AWS Configuration
AWS_REGION = "us-east-1"
MODEL_ID = "anthropic.claude-sonnet-4-5-20250929-v1:0"

# Initialize Bedrock client to verify connectivity
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION
)

print(f"✓ AWS Bedrock configured for region: {AWS_REGION}")
print(f"✓ Using model: {MODEL_ID}")

# Supply the pre-installed polciy and guardrail IDs
ARC_POLICY_ARN = "arn:aws:bedrock:us-east-1:505766489067:automated-reasoning-policy/413037g0my75"
GUARDRAIL_ID = "aq9ga2k7be2c"
GUARDRAIL_VERSION = "DRAFT"
KNOWLEDGE_BASE_ID = "2KSYYY00BD"
# NOTE: the default model for Strands is us.anthropic.claude-sonnet-4-20250514-v1:0
# MODEL_ID = "us.amazon.nova-lite-v1:0"
# MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"


# Setup the environment for the agent and tool
# Allow for the metadata to be retrieved on sources from the KB
os.environ['RETRIEVE_ENABLE_METADATA_DEFAULT'] = 'true'
# Allow for the retrieve tool to interact with the KB
os.environ['KNOWLEDGE_BASE_ID'] = KNOWLEDGE_BASE_ID



# Define a notification hook to listen to events and then process the result and call
# Automated Reasoning attached via the Guardrail and report on the findings.  This
# can be used possibly re-write the output or add a flag on if the output is correct.
# Define a notification hook to listen to events and then process the result and call
# Automated Reasoning attached via the Guardrail and report on the findings.  This
# can be used possibly re-write the output or add a flag on if the output is correct.
class NotifyOnlyGuardrailsHook(HookProvider):
    
    def __init__(self, guardrail_id: str, guardrail_version: str, arc_policy_arn: str):
        self.guardrail_id = guardrail_id
        self.guardrail_version = guardrail_version
        self.arc_policy_arn = arc_policy_arn
        self.bedrock_client = boto3.client("bedrock-runtime")
        self.input = ''
        self.claim_valid = True
        self.findings = ''
        self.policy_definition = {}
        self.before_tool_event_flag = False
        self.before_model_event_flag = False

        if self.arc_policy_arn:
            try:
                bedrock_client = boto3.client('bedrock')
                response = bedrock_client.export_automated_reasoning_policy_version(policyArn=self.arc_policy_arn)
                self.policy_definition = response.get('policyDefinition', {})
            except Exception as e:
                print(f"Error getting policy definition: {str(e)}")
                raise

    def register_hooks(self, registry: HookRegistry) -> None:
        registry.add_callback(BeforeModelCallEvent, self.before_model_event)
        registry.add_callback(BeforeToolCallEvent, self.before_tool_event)
        registry.add_callback(MessageAddedEvent, self.message_added)

    def message_added(self, event: MessageAddedEvent) -> None:
        if self.before_tool_event_flag:
            # Since a tool was called, just ignore this message addition
            self.before_tool_event_flag = False
            return
        
        # Get the content
        content = "".join(block.get("text", "") for block in event.message.get("content", []))

        # Determine the source
        if event.message.get("role") == "user":
            # Store the input for later usage and allow the loop to continue to process
            self.input = content
            return

        if not content:
            return
            #do something 

        # Capture if this is the first time that findings will be created
        first_findings = (not self.findings)

        # Format a request to send to the guardrail
        content_to_validate = [
            {"text": {"text": self.input, "qualifiers": ["query"]}},
            {"text": {"text": content, "qualifiers": ["guard_content"]}}
        ]
        
        # Call the guardrail
        response = self.bedrock_client.apply_guardrail(
            guardrailIdentifier=self.guardrail_id,
            guardrailVersion=self.guardrail_version,
            source="OUTPUT",
            content=content_to_validate
        )

        # Determine if the output is correct
        self.findings = extract_reasoning_findings(response, self.policy_definition)
         
        assessments = response.get("assessments", [])
        if assessments and len(assessments):
            self.claim_valid = False

        # Add information to the output
        if self.findings and first_findings:
            new_output = content
            new_output = new_output + f"\n***FINDINGS: ***:\n{self.findings}"
            new_output = new_output + f"\n***CLAIM VALID: ***:\n{self.claim_valid}"
            event.message["content"][0]["text"] = new_output
        
    def before_model_event(self, event: BeforeModelCallEvent) -> None:
        self.before_model_event_flag = True

    def before_tool_event(self, event: BeforeToolCallEvent) -> None:
        self.before_tool_event_flag = True

# Create structured output
class StructuredOutputModel(BaseModel):
    claim_valid: bool
    content: str
    findings: str

# Provide the config for botocore
boto_config = BotocoreConfig(
    retries={"max_attempts": 3, "mode": "standard"},
    connect_timeout=5,
    read_timeout=60
)

# Create a Bedrock model with guardrail configuration
bedrock_model = BedrockModel(
    boto_client_config=boto_config,
    model_id=MODEL_ID,
    # NOTE: An alternative option is to supply the guardrail here.  If going that route, the ARc findings aren't present.
    # To ensure that the findings are present and can be used to re-write the output, rely on a hook
)

agent_instructions="""You are an expert automotive claims validaiton specialist that determines if the users auto insurance claim is valid based on the provided information and details within the policy contract.
    
You will be provided with JSON data that has claim information and vehicle damage information, you should:
1. Extract from the JSON data required claim information to be validated
2. Focus on time of event and time of claim creation
3. Focus on claims and coverage inconsistencies

Your responses should :
- If your response is "Valid claim", then output the provide the full JSON structure provided from the input
- If you response is "Invalid claim", then output the response "This Claim is Invalid and no appraisal nor settlement is required"
- If you response is "Invalid claim", then provide clear explanation on why is invalid in the output
- In cases where a clear outcome is not present, recommend the user to check with their insurance agent directly. 

Take your time to think though the answer and evalute carefully."

"""


# Create agent with the guardrail-protected model
agent = Agent(
    name="Claims Validator - ARC",
    description="A Single agent with Claims Validation tools capabilities",
    model=bedrock_model,
    hooks=[NotifyOnlyGuardrailsHook(GUARDRAIL_ID, GUARDRAIL_VERSION, ARC_POLICY_ARN)],
    tools=[retrieve],
    system_prompt=agent_instructions
)



################# A2A ################
app = FastAPI()
runtime_url = os.environ.get('AGENTCORE_RUNTIME_URL', 'http://127.0.0.1:9000/')
host, port = "0.0.0.0", 9000

a2a_server = A2AServer(
    agent=agent,
    http_url=runtime_url,
    serve_at_root=True,
    
)

@app.get("/ping")
def ping():
    return {"status": "healthy"}

# @app.on_event("startup")
# async def startup_event():
#     """Initialize MCP client on startup"""
#     await setup_agent_tools()

app.mount("/", a2a_server.to_fastapi_app())



if __name__ == "__main__":
    uvicorn.run(app, host=host, port=port)

################# A2A ################

Overwriting agents/claimvalidation-agent.py


Let's write a requirements.txt file with dependencies that are needed for the agent.

In [18]:
%%writefile agents/requirements.txt
boto3==1.40.50
bedrock-agentcore==0.1.7
strands-agents[a2a]
strands-agents-tools
pyyaml
ddgs

Overwriting agents/requirements.txt


###  Deploy to AgentCore Runtime

#### Setup Cognito User Pool

In [19]:
from helpers.utils import setup_cognito_user_pool, reauthenticate_user

print("Setting up Amazon Cognito user pool...")
cognito_config = (
    setup_cognito_user_pool()
)  # You'll get your bearer token from this output cell.
print("Cognito setup completed ✓")

Setting up Amazon Cognito user pool...
Pool id: us-east-1_RYybrxkt5
Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_RYybrxkt5/.well-known/openid-configuration
Client ID: vt3fjviqqonlkui98f3gndkj5
Bearer Token: eyJraWQiOiJBQUEzMUZhTlArMHc1QWc4cnlwbkQzamJUbDJ4VVBOWURWWWJpVGdwVTBnPSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiI0NDU4ZDRhOC1hMDcxLTcwZTAtMGFiOS1mYTEyN2JlNzA2Y2UiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV9SWXlicnhrdDUiLCJjbGllbnRfaWQiOiJ2dDNmanZpcXFvbmxrdWk5OGYzZ25ka2o1Iiwib3JpZ2luX2p0aSI6IjQ1OTdiYzMyLWM1NzgtNDUxMi1hNDIxLTc1MzgyZjI5ZWFmYiIsImV2ZW50X2lkIjoiMzBmYmY3ZDMtZjU3Mi00ZWE2LWI3YTYtN2E5NDJhOWI0YjY5IiwidG9rZW5fdXNlIjoiYWNjZXNzIiwic2NvcGUiOiJhd3MuY29nbml0by5zaWduaW4udXNlci5hZG1pbiIsImF1dGhfdGltZSI6MTc3MDI0ODg1NSwiZXhwIjoxNzcwMjUyNDU1LCJpYXQiOjE3NzAyNDg4NTUsImp0aSI6IjM1Zjk5MDUxLTAxYTUtNGNlMi04M2MyLWFmOTIzYTU2ZDZkMiIsInVzZXJuYW1lIjoidGVzdHVzZXIifQ.NpeIjJ2XqVgXyrnqB0O5GBfYsyrzk1z-f5hXfUFJNmRvCnOorstdUJQ5XeavbWr9zqRH7x580Lci2I

#### Create IAM Role for the Agents

In [20]:
# from helpers.utils import create_agentcore_runtime_execution_role, AWS_CLAIMSVALIDATION_ROLE_NAME

# execution_role_arn_claimvalidation = create_agentcore_runtime_execution_role(AWS_CLAIMSVALIDATION_ROLE_NAME)

In [21]:
from helpers.utils import create_agentcore_runtime_execution_role, AWS_FNOL_ROLE_NAME

execution_role_arn= create_agentcore_runtime_execution_role(AWS_FNOL_ROLE_NAME)

ℹ️ Role AWSFNOLAssistantBedrockAgentCoreRole-us-east-1 already exists
Role ARN: arn:aws:iam::505766489067:role/AWSFNOLAssistantBedrockAgentCoreRole-us-east-1


In [22]:
execution_role_arn

'arn:aws:iam::505766489067:role/AWSFNOLAssistantBedrockAgentCoreRole-us-east-1'

##### Configure and deploy our  agent:

In [23]:
from bedrock_agentcore_starter_toolkit import Runtime

agentcore_runtime_claimvalidation_agent = Runtime()
claimvalidation_agent_name="aws_claimvalidation_assistant"

region = boto_session.region_name

# Configure the deployment
response_claimvalidation_agent = agentcore_runtime_claimvalidation_agent.configure(
    entrypoint="agents/claimvalidation-agent.py",
    # execution_role=execution_role_arn_claimvalidation,
    execution_role=execution_role_arn,
    # auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="agents/requirements.txt",
    region=region,
    agent_name=claimvalidation_agent_name,
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_config.get("client_id")],
            "discoveryUrl": cognito_config.get("discovery_url"),
        }
    },
    protocol="A2A",
)

print("Configuration completed:", response_claimvalidation_agent)




Entrypoint parsed: file=/home/ec2-user/SageMaker/agents/claimvalidation-agent.py, bedrock_agentcore_name=claimvalidation-agent
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: aws_claimvalidation_assistant
Memory disabled
Network mode: PUBLIC


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

📄 Using existing Dockerfile: /home/ec2-user/SageMaker/Dockerfile

Generated .dockerignore: /home/ec2-user/SageMaker/.dockerignore
Keeping 'aws_claimvalidation_assistant' as default agent
Bedrock AgentCore configured: /home/ec2-user/SageMaker/.bedrock_agentcore.yaml


Configuration completed: config_path=PosixPath('/home/ec2-user/SageMaker/.bedrock_agentcore.yaml') dockerfile_path=PosixPath('/home/ec2-user/SageMaker/Dockerfile') dockerignore_path=PosixPath('/home/ec2-user/SageMaker/.dockerignore') runtime='Docker' runtime_type=None region='us-east-1' account_id='505766489067' execution_role='arn:aws:iam::505766489067:role/AWSFNOLAssistantBedrockAgentCoreRole-us-east-1' ecr_repository=None auto_create_ecr=True s3_path=None auto_create_s3=False memory_id=None network_mode='PUBLIC' network_subnets=None network_security_groups=None network_vpc_id=None


In [24]:
launch_result_claimvalidation = agentcore_runtime_claimvalidation_agent.launch()
print("Launch completed:", launch_result_claimvalidation.agent_arn)

claimvalidation_agent_arn = launch_result_claimvalidation.agent_arn

🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'aws_claimvalidation_assistant' to account 505766489067 (us-east-1)
Generated image tag: 20260204-234752-390
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: aws_claimvalidation_assistant
ECR repository available: 505766489067.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-aws_claimvalidation_assistant
Using execution role from config: arn:aws:iam::505766489067:role/AWSFNOLAssistantBedrockAgentCoreRole-us-east-1
Preparing CodeBuild project and uploading source...


✅ Reusing existing ECR repository: 505766489067.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-aws_claimvalidation_assistant


Getting or creating CodeBuild execution role for agent: aws_claimvalidation_assistant
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-1db4e090aa
Reusing existing CodeBuild execution role: arn:aws:iam::505766489067:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-1db4e090aa
Using dockerignore.template with 47 patterns for zip filtering
Uploaded source to S3: aws_claimvalidation_assistant/source.zip
Updated CodeBuild project: bedrock-agentcore-aws_claimvalidation_assistant-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.0s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 7.3s
🔄 DOWNLOAD_SOURCE started (total: 8s)
✅ DOWNLOAD_SOURCE completed in 1.0s
🔄 BUILD started (total: 9s)
✅ BUILD completed in 16.6s
🔄 POST_BUILD started (total: 26s)
✅ POST_BUILD completed in 13.5s
🔄 COMPLETED started (total: 39s)
✅ COMPLETED completed in 1.0s
🎉 CodeBuild completed successful

Launch completed: arn:aws:bedrock-agentcore:us-east-1:505766489067:runtime/aws_claimvalidation_assistant-4cWJdn9uW8


In [25]:
status_response = agentcore_runtime_claimvalidation_agent.status()
status = status_response.endpoint["status"]

print(f"Final status: {status}")

Retrieved Bedrock AgentCore status for: aws_claimvalidation_assistant


Final status: READY


#### Export and save outputs

Export variables to be used in next notebooks:

In [26]:
CLAIMSVALIDATION_AGENT_ID = launch_result_claimvalidation.agent_id
CLAIMSVALIDATION_AGENT_ARN = launch_result_claimvalidation.agent_arn
CLAIMSVALIDATION_AGENT_NAME = claimvalidation_agent_name


CLAIMSVALIDATION_COGNITO_CLIENT_ID = cognito_config.get("client_id")
CLAIMSVALIDATION_COGNITO_SECRET = cognito_config.get("client_secret")
CLAIMSVALIDATION_DISCOVERY_URL = cognito_config.get("discovery_url")

%store CLAIMSVALIDATION_AGENT_ID
%store CLAIMSVALIDATION_AGENT_ARN
%store CLAIMSVALIDATION_AGENT_NAME
%store CLAIMSVALIDATION_COGNITO_CLIENT_ID
%store CLAIMSVALIDATION_COGNITO_SECRET
%store CLAIMSVALIDATION_DISCOVERY_URL

Stored 'CLAIMSVALIDATION_AGENT_ID' (str)
Stored 'CLAIMSVALIDATION_AGENT_ARN' (str)
Stored 'CLAIMSVALIDATION_AGENT_NAME' (str)
Stored 'CLAIMSVALIDATION_COGNITO_CLIENT_ID' (str)
Stored 'CLAIMSVALIDATION_COGNITO_SECRET' (str)
Stored 'CLAIMSVALIDATION_DISCOVERY_URL' (str)


In [27]:
# from helpers.utils import put_ssm_parameter, SSM_CLAIMSVALIDATION_AGENT_ARN

# put_ssm_parameter(SSM_CLAIMSVALIDATION_AGENT_ARN, CLAIMSVALIDATION_AGENT_ARN)

In [28]:
from helpers.utils import put_ssm_parameter, SSM_FNOL_AGENT_ARN

put_ssm_parameter(SSM_FNOL_AGENT_ARN, launch_result_claimvalidation.agent_arn)

### Invoking A2A agents

Firstly, let's refresh the auth token:

In [29]:
bearer_token = reauthenticate_user(
    cognito_config.get("client_id"), 
    cognito_config.get("client_secret")
)

In [30]:
cognito_config.get("client_id")

'vt3fjviqqonlkui98f3gndkj5'

In [31]:
CLAIMSVALIDATION_COGNITO_CLIENT_ID

'vt3fjviqqonlkui98f3gndkj5'

In [32]:
import logging
from uuid import uuid4
from urllib.parse import quote

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

def fetch_agent_card(agent_arn):
    # URL encode the agent ARN
    escaped_agent_arn = quote(agent_arn, safe='')

    # Construct the URL
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/.well-known/agent-card.json"
    logger.info(url)
    # Generate a unique session ID
    session_id = str(uuid4())
    logger.info(f"Generated session ID: {session_id}")

    # Set headers
    headers = {
        'Accept': '*/*',
        'Authorization': f'Bearer {bearer_token}',
        'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id,
        'X-Amzn-Trace-Id': f'aws_docs_assistant_{session_id}'
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse and pretty print JSON
        agent_card = response.json()
        logger.info(json.dumps(agent_card, indent=2))

        return agent_card

    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching agent card: {e}")
        return None

In [33]:
claimvalidation_agent_arn

'arn:aws:bedrock-agentcore:us-east-1:505766489067:runtime/aws_claimvalidation_assistant-4cWJdn9uW8'

In [34]:
fetch_agent_card(claimvalidation_agent_arn)

ERROR:__main__:Error fetching agent card: 424 Client Error: Failed Dependency for url: https://bedrock-agentcore.us-east-1.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aus-east-1%3A505766489067%3Aruntime%2Faws_claimvalidation_assistant-4cWJdn9uW8/invocations/.well-known/agent-card.json


#### Test agents

Now, let's invoke the first agent, using A2A:

In [ ]:
# Ask the agent a question
auto_policy_agent("I was delivering goods for my company using my personal car. I got into an accident, this was over 3 months ago. Does my policy cover this?")

In [ ]:
# Get the structured output.  Without providing a prompt, the agent will act on conversation history to get the response
response = auto_policy_agent.structured_output(StructuredOutputModel)

# Extract the findings from the response
print(f"response.claim_valid = {response.claim_valid}")
# print(f"repsonse.content = {response.content}")
# print(f"repsonse.findings = {response.findings}")

In [ ]:
# Extract the findings from the response
# print(f"response.claim_valid = {response.claim_valid}")
print(f"repsonse.content = {response.content}")
# print(f"repsonse.findings = {response.findings}")

In [ ]:
# Extract the findings from the response
# print(f"response.claim_valid = {response.claim_valid}")
# print(f"repsonse.content = {response.content}")
print(f"repsonse.findings = {response.findings}")